# TAREA 3
## Importar librerías y módulos de Python
##### Importamos las librerías/módulos específicos de la siguiente forma.

In [ ]:
# Importacion.
# import librosa
from scipy.io import wavfile
import IPython
import os
import numpy as np
import matplotlib.pyplot as plt

## Especificar directorios de entrada y salida
##### Aquí definimos los directorios donde guardaremos los audios con los que vamos a trabajar, así como dónde se van a guardar aquellos que generamos a lo largo de la práctica.


In [ ]:
# Directorios que usaremos.
cwd = os.getcwd()
audio_input_path = os.path.join(cwd, os.path.join('audio','examples'))  # 'examples'
audio_output_path = os.path.join(cwd, os.path.join('audio','_output'))
print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

## Cargar el archivo de audio
##### Cargamos el archivo de audio .wav en este caso.

In [ ]:
# Cargamos el archivo de audio.
filename = os.path.join(audio_input_path, 'the_last_of_us_reduced.wav')
# audio_data, sample_rate = librosa.load(filename, sr=None, mono=False)
sample_rate, audio_data = wavfile.read(filename)
tamaño_bytes = os.path.getsize(filename)  # Obtiene el tamaño en bytes
tamaño_mb = tamaño_bytes / (1024 * 1024)     # Convierte a megabytes
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')
print(f'Número de canales: {audio_data.shape[1]}')
print(f"El tamaño del audio es: {tamaño_mb:.2f} MB")


##### Vamos a escucharlo. Para que esto se haga correctamente, hay que indicarle la frecuencia de muestreo.

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo

## Conversión de estereo a mono
##### Ahora, por simplificación, vamos a calcular la media por canal para obtener un sonido mono.

In [ ]:
# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Column-wise.
# Mantenemos la misma resolucion que antes.
new_data_mono = new_data_mono.astype(np.int16)

tamaño_bytes = new_data_mono.nbytes  # Tamaño en bytes del array
tamaño_mb = tamaño_bytes / (1024 * 1024)
print('Nuevos datos de audio (mono):')
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')
print(f'Número de canales: {new_data_mono[:5]}...')
print(f"El tamaño del audio es: {tamaño_mb:.2f} MB")

##### Vamos a guardarlo

In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'the_last_of_us_reduced.wav'),
    rate=sample_rate,
    data=new_data_mono
)

##### Vamos a escucharlo de nuevo

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

## Mostrar una gráfica en el tiempo para ambos audios

In [ ]:
ampl_estereo = len(audio_data)
ampl_mono = len(new_data_mono)
print(f'Número de muestras del audio con fs=48 kHz (valores de amplitud): {ampl_estereo}')
print(f'Número de muestras del audio con fs=24 kHz (valores de amplitud): {ampl_mono}')

In [ ]:
# Construimos el array para el eje x que representa el tiempo de la grabación.
# Tiene la forma: np.arange(Vi, Vf, P).
t1 = np.arange(0, ampl_estereo/sample_rate, 1/sample_rate)
t2 = np.arange(0, ampl_mono/sample_rate, 1/sample_rate)

In [ ]:
print(t1)
print(t2)

In [ ]:
# Creamos la figura.
fig, ax = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Solo mostramos las primeras 50 muestras de amplitud (por claridad).
end = 50

# Señal a 44 kHz.
ax[0].plot(t1[:end], audio_data[:end], marker='X')
ax[0].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[0].set_ylabel('Amplitud')
ax[0].grid(True)

# Señal a 44 kHz.
# Utilizamos ratio para ajustar el eje x de ambas gráficas
# ya que la fs es menor en esta señal.
ratio = sample_rate / sample_rate  
ax[1].plot(t2[:int(end/ratio)], new_data_mono[:int(end/ratio)], c='tab:red', marker='X')
ax[1].set_title(f'Audio en el dominio del tiempo muestreado a {sample_rate} Hz')
ax[1].set_xlabel('Tiempo (s)')
ax[1].set_ylabel('Amplitud')
ax[1].grid(True)

# Mostramos la figura.
plt.tight_layout()
plt.show()


## Explicación
### Ahora se va a explicar las diferencias entre audio estéreo y mono

##### - El audio estéreo usa dos canales para crear una sensación de espacio: podemos notar sonidos que vienen más de un lado u otro. 
##### - El audio mono, en cambio, usa un solo canal, todo suena “en el centro”. 
##### Básicamente, estéreo da sensación de direcciones, mientras que mono es más sencillo y ocupa menos espacio, pero sin efecto de ubicación de los sonidos.